### Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Import Pixar csv

In [2]:
pixar_df = pd.read_csv('../data/pixar__studio12.csv')
pixar_df.sample(5)

,Moviename,Releaseyear,Budget(million),US and canada(million),Other territories(million),Worldwide(million),Rotten Tomatoes
4,Finding Nemo,2003,940,3397,5313,8710,99%
18,Coco,2017,1750,2097,5974,8071,97%
12,Brave,2012,1850,2373,3017,5390,78%
20,Toy Story 4,2019,2000,4340,6394,10734,97%
8,WALL-E,2008,1800,2238,2975,5213,95%


## Scrapping wiki table

In [3]:
wiki = 'https://en.wikipedia.org/wiki/List_of_Pixar_films'

In [4]:
res = requests.get(wiki)
soup = BeautifulSoup(res.content, "html.parser")

In [5]:
wrappers = soup.find_all('table', {'class':'wikitable'})
wrappers[4]

<table class="wikitable sortable" style="text-align:center; margin=auto; font-size:90%">
<tbody><tr>
<th rowspan="2" scope="col">Film
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Picture" title="Academy Award for Best Picture">Best Picture</a>
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Animated_Feature" title="Academy Award for Best Animated Feature">Animated Feature</a>
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Original_Screenplay" title="Academy Award for Best Original Screenplay">Original Screenplay</a>
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Adapted_Screenplay" title="Academy Award for Best Adapted Screenplay">Adapted Screenplay</a>
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Original_Score" title="Academy Award for Best Original Score">Original Score</a>
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Original_Son

### Academy Awards Table

    Column names extraction

In [6]:
col_names = wrappers[4].find_all('th')
col_names = [i.getText().strip() for i in col_names]
col_names.remove('Sound[a]')
col_names = [i.lower().replace(' ', '_') for i in col_names]
col_names.remove('other')
col_names.append('other')
col_names

['film',
 'best_picture',
 'animated_feature',
 'original_screenplay',
 'adapted_screenplay',
 'original_score',
 'original_song',
 'sound_editing',
 'sound_mixing',
 'other']

    Rows extraction

In [7]:
def extract_wiki_table(rows, columns):
    # dictionary to control rowspan values in each iteration
    rowspan_values = {col: 1 for col in columns}
    data = []

    for idx, row in enumerate(rows):
        # list_ will be the list of values for each row in the table
        list_ = []
        cells = row.find_all('td')
        count_cells = 0
        for i in range(len(columns)):
            # check if last_row[column] rowspan is greater than 1 to pick the value of that cell
            # update rowspan of this row decreasing in 1
            if int(rowspan_values[columns[i]]) > 1:
                last_row = data[idx - 1]
                last_value = last_row[i]
                list_.append(last_value)
                rowspan_values[columns[i]] -= 1
            else:
                list_.append(cells[count_cells].getText().strip().replace('\\n', ''))
                if cells[count_cells].get('rowspan'):
                    rowspan_values[columns[i]] = int(cells[count_cells].get('rowspan'))
                else:
                    rowspan_values[columns[i]] = 1
                count_cells += 1
        data.append(list_)
    
    return pd.DataFrame(data, columns=columns)

In [8]:
rows = wrappers[4].find_all('tr')
rows = rows[2::]

# Excluding 4 last rows to extract the info
rows = rows[:22]
df_1 = extract_wiki_table(rows, col_names)
df_1.sample(3)

,film,best_picture,animated_feature,original_screenplay,adapted_screenplay,original_score,original_song,sound_editing,sound_mixing,other
1,A Bug's Life,,Award not yet introduced,,Ineligible,Nominated,,,,
8,WALL-E,,Won,Nominated,Ineligible,Nominated,Nominated,Nominated,Nominated,
14,Inside Out,,Won,Nominated,Ineligible,,,,,


    The last 4 movies in the table from wikipedia were added by hand because there was a problem with the columns, which did not match.

In [9]:
remaining_movies = [
    ['Soul', '', 'Won', '', 'Ineligible', 'Won', '', 'Nominated', 'Nominated', ''],
    ['Luca', '', 'Nominated', '', 'Ineligible', '', '', '', '', ''],
    ['Turning Red', '', 'Nominated', '', 'Ineligible', '', '', '', '', ''],
    ['Lightyear', '', '', 'Ineligible', '', '', '', '', '', '']
]
df_2 = pd.DataFrame(remaining_movies, columns=col_names)
df_2

,film,best_picture,animated_feature,original_screenplay,adapted_screenplay,original_score,original_song,sound_editing,sound_mixing,other
0,Soul,,Won,,Ineligible,Won,,Nominated,Nominated,
1,Luca,,Nominated,,Ineligible,,,,,
2,Turning Red,,Nominated,,Ineligible,,,,,
3,Lightyear,,,Ineligible,,,,,,


    The two data frames were then concatenated to merge them into one.

In [10]:
academy_awards_df = pd.concat([df_1, df_2], ignore_index=True)

### Box Office Table

    Column names extraction

In [11]:
col_names = wrappers[2].find_all('th')
col_names = [i.getText().strip() for i in col_names]
col_names.remove('Box office gross')
col_names = [i.lower().replace(' ', '_') for i in col_names]
col_names.remove('ref.')
col_names.append('ref.')
col_names

['year',
 'film',
 'budget',
 'u.s._and_canada',
 'other_territories',
 'worldwide',
 'ref.']

    Rows extraction

In [12]:
rows = wrappers[2].find_all('tr')
rows = rows[2::]
df = extract_wiki_table(rows, col_names)
df = df.drop(columns=["ref."])

In [13]:
final_df = pd.merge(academy_awards_df, df, on="film", how="outer")
final_df.to_csv('../data/pixar_academy_awards.csv', index=False)